In [12]:
import time
from os import register_at_fork
import sys
import numpy as np
import random

# Pick random move
def randomPlayer(board):
    m = randint(8)
    while noRoomInColumn(m,board):                # no move in this column, try again
        m = randint(8)                            
    return m

# Board is 8x8 numpy array
# 0 = no piece
# 1 = X piece
# 2 = O piece

blank = 0
X = 1
O = 2

symbol = [' ','X','O']

N = 8      

# Creates a fresh empty board
def getEmptyBoard():                              
    return np.zeros((N,N)).astype(int)

# This will be used to indicate an error when a column is full

ERROR = -1

# Check for error: since numpy arrays work strangely with comparisons

def isError(B):
    if type(B) == int:
        return B == ERROR
    else:
        return False

# Print out a human-readable version of the board

def printBoard(B,ind=0):
    indent = '\t'*ind
    if isError(B):
        print(indent,"ERROR: Overflow in column.")
        return
    print(indent,'  0 1 2 3 4 5 6 7')
    print(indent,'-------------------')
    for row in range(N):
        print(indent,'|',end='')
        for col in range(N):
            print(' '+ symbol[B[row][col]],end='')
        print(' |')
    print(indent,'-------------------')


def illegalMove(m):
    return not(0 <= m <= 7)

def noRoomInColumn(move,board):
    return board[0][move] != blank            

# Check if a win has occured
def checkWin(player, row, col, board):    
    #check horizontal
    horiz = checkNHoriz(player, row, col, board)
    if horiz[0] >= 4:
      return horiz[1]

    # vertical
    vert = checkNVert(player, row, col, board)
    if vert[0] >= 4:
      return vert[1]

    # diagonals
    diagLeft = checkNDiagLeft(player, row, col, board)
    if diagLeft[0] >= 4:
      return diagLeft[1]

    diagRight = checkNDiagRight(player, row, col, board)
    if diagRight[0] >= 4:
      return diagRight[1]
        
    return 0

OWIN = sys.maxsize
XWIN = -OWIN

THREE_SCORE = 50                 
TWO_SCORE = 10                    

# Return evaluation of a move on the left diagonal from a player's point of view
def checkNDiagLeft(player, i, j, board):
  firstPlayer = blank
  if illegalMove(i + 3) or illegalMove(j - 3):
    return [0, firstPlayer]
  else:
    ct = 0
    for x in range(0, 4):
      if board[i + x][j - x] != blank:
        if firstPlayer == blank:
          firstPlayer = board[i + x][j - x]
          ct+=1
        elif board[i + x][j - x] != firstPlayer:

          return [0, firstPlayer]

        elif board[i + x][j - x] == firstPlayer:
          ct+=1
    return [ct, firstPlayer]


# Return evaluation of a move on the right diagonal from a player's point of view
def checkNDiagRight(player, i, j, board):
  firstPlayer = blank
  if illegalMove(i + 3) or illegalMove(j + 3):
    return [0, firstPlayer]
  else:
    ct = 0
    for x in range(0, 4):
      if board[i + x][j + x] != blank:
        if firstPlayer == blank:
          firstPlayer = board[i + x][j + x]
          ct+=1
        elif board[i + x][j + x] != firstPlayer:

          return [0, firstPlayer]
        elif board[i + x][j + x] == firstPlayer:
          ct+=1
    return [ct, firstPlayer]

# Return evaluation of a move on the vertical from a player's point of view
def checkNVert(player, i, j, board):
  firstPlayer = blank
  if illegalMove(i + 3):
    return [0, firstPlayer]
  else:

    ct = 0
    for x in range(0, 4):
      if board[i + x][j] != blank:
        if firstPlayer == blank:
          firstPlayer = board[i + x][j]
          ct+=1
        elif board[i + x][j] != firstPlayer:
          return [0, firstPlayer]
        elif board[i + x][j] == firstPlayer:
          ct+=1

    return [ct, firstPlayer]

# Return evaluation of a move on the horizontal from a player's point of view
def checkNHoriz(player, i, j, board):
  firstPlayer = blank
  if illegalMove(j + 3):
    return [0, firstPlayer]
  else:
    ct = 0
    for x in range(0, 4):
      if board[i][j + x] != blank:
        if firstPlayer == blank:
          firstPlayer = board[i][j + x]
          ct+=1
        elif board[i][j + x] != firstPlayer:
          return [0, firstPlayer]
        elif board[i][j + x] == firstPlayer:
          ct+=1

    return [ct, firstPlayer]

# Board evaluation function
def eval(board):
    numTwosX = 0
    numThreesX = 0
    numTwosO = 0
    numThreesO = 0
    score = 0
    for i in range(len(board)):
      for j in range(len(board[0])):
          #horizontal
          ct = checkNHoriz(board[i][j], i, j, board)
          if ct[1] == X:
            if ct[0] == 2:
              numTwosX +=1
            elif ct[0] == 3:
              numThreesX +=1
            elif ct[0] == 4:
              return XWIN
          elif ct[1] == O:
            if ct[0] == 2:
              numTwosO +=1
            elif ct[0] == 3:
              numThreesO +=1
            elif ct[0] == 4:
              return OWIN
          

          ct = checkNVert(board[i][j], i, j, board)
          if ct[1] == X:
            if ct[0] == 2:
              numTwosX +=1
            elif ct[0] == 3:
              numThreesX +=1
            elif ct[0] == 4:
              return XWIN
          elif ct[1] == O:
            if ct[0] == 2:
              numTwosO +=1
            elif ct[0] == 3:
              numThreesO +=1
            elif ct[0] == 4:
              return OWIN

          ct = checkNDiagLeft(board[i][j], i, j, board)
          if ct[1] == X:
            if ct[0] == 2:
              numTwosX +=1
            elif ct[0] == 3:
              numThreesX +=1
            elif ct[0] == 4:
              return XWIN
          elif ct[1] == O:
            if ct[0] == 2:
              numTwosO +=1
            elif ct[0] == 3:
              numThreesO +=1
            elif ct[0] == 4:
              return OWIN

          ct = checkNDiagRight(board[i][j], i, j, board)
          if ct[1] == X:
            if ct[0] == 2:
              numTwosX +=1
            elif ct[0] == 3:
              numThreesX +=1
            elif ct[0] == 4:
              return XWIN
          elif ct[1] == O:
            if ct[0] == 2:
              numTwosO +=1
            elif ct[0] == 3:
              numThreesO +=1
            elif ct[0] == 4:
              return OWIN

    xscore = numTwosX * 10 + numThreesX * 50
    oscore = numTwosO * 10 + numThreesO * 50

    score = oscore - xscore

    return score     

maxNodeLimit = 100000    

countNodes = 0

# drop a piece into the board at some position
def dropPiece(player,move,board):
    if illegalMove(move):
        return -1
    elif noRoomInColumn(move, board):
        return -1
    else:
        i = 0
        while board[i][move] == 0:
          if i == len(board) - 1:
            board[i][move] = player
            return i
          i+=1
        board[i - 1][move] = player
    return i - 1         

# minmax algorithm 
def minMax(board, player, depth, alpha, beta):
  if player == O:
    maxVal = XWIN
    best = -1
    for i in range(len(board)):
      if not noRoomInColumn(i, board):
        row = dropPiece(player, i, board)
        val = minMaxHelper(board, X, depth + 1, XWIN, OWIN)
        if val == None:
          print("node limit reached: " + int(countNodes))
        if val > maxVal:
          best = i
          maxVal = val
        board[row][i] = 0
    if best == -1:
      best = randomPlayer(board)
    return maxVal, best
  return -1


def minMaxHelper(board, player, depth, alpha, beta):
   
    global countNodes
    countNodes += 1  
    
    if countNodes > maxNodeLimit:
        return None

    nomoves = True
    for i in range(len(board)):
        nomoves = nomoves and noRoomInColumn(i, board)
    if nomoves or depth == maxDepth:
      return eval(board)


    elif player == O:
      val = XWIN
      for j in range(len(board)):
        if not noRoomInColumn(j,board):
          alpha = max(val, alpha)
          if beta < alpha: break
          row = dropPiece(O, j, board)
          val = max(val, minMaxHelper(board, X, depth + 1, alpha, beta))
          board[row][j] = 0
      return val
    else:
      val = OWIN
      for j in range(len(board)):
        if not noRoomInColumn(j,board):
          beta = min(val, beta)
          if beta < alpha: break
          row = dropPiece(X, j, board)
          val = min(val, minMaxHelper(board, O, depth + 1, alpha, beta))
          board[row][j] = 0
      return val

def player(board):    
    (_,move) = minMax(board,O,0,-sys.maxsize,sys.maxsize)  
    return move

maxDepth = 4


# Play against the bot
def playBot():
  p = 1
  board = getEmptyBoard()
  printBoard(board)

  for k in range(64):
      if p == 1:
        move = int(input('X\'s move: '))  # convert string to int
        if illegalMove(move):
            print("Illegal move: not in range 0..7.")
            break
        else:
            print("You entered",move)
            row = dropPiece(p, move, board)
            printBoard(board)
            win = None
            win = checkWin(p, row, move, board)
            if win:
              print("X Wins")
              break
        p +=1
      else:
        t_start = time.perf_counter()
        move = player(board)
        t_end = time.perf_counter()

        countNodes = 0

        print("Time elapsed:", np.around(t_end-t_start,2), "secs.") 
        print("Bot played " + str(move))
        row = dropPiece(p, move, board)
        printBoard(board)
        win = None
        win = checkWin(p, row, move, board)
        if win:
          print("O Wins")
          break
        p-=1
    

playBot()




   0 1 2 3 4 5 6 7
 -------------------
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 -------------------
X's move: 0
You entered 0
   0 1 2 3 4 5 6 7
 -------------------
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 | X               |
 -------------------
Time elapsed: 0.89 secs.
Bot played 2
   0 1 2 3 4 5 6 7
 -------------------
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 | X   O           |
 -------------------
X's move: 1
You entered 1
   0 1 2 3 4 5 6 7
 -------------------
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 |                 |
 | X X O           |
 --------------

KeyboardInterrupt: ignored